In [1]:
from molsysmt.native import CIFFileHandler
import pickle

handler = CIFFileHandler('components.cif')
cif_entries = handler.parse()

In [2]:
types_dict = {}

for ii in cif_entries:
    jj = cif_entries[ii]['_chem_comp']['type']
    if jj in types_dict:
        types_dict[jj].append(ii)
    else:
        types_dict[jj] = [ii]

types = ['"L-PEPTIDE LINKING"',
         '"D-PEPTIDE LINKING"',
         '"L-peptide linking"',
         '"D-peptide linking"',
         '"PEPTIDE LINKING"',
         '"peptide linking"',
         '"L-peptide NH3 amino terminus"',
         '"L-peptide COOH carboxy terminus"',
         '"L-PEPTIDE COOH CARBOXY TERMINUS"',
         '"D-PEPTIDE NH3 AMINO TERMINUS"',
         '"D-peptide NH3 amino terminus"']

data = []
for ii in types:
    data += types_dict[ii]



In [3]:
output = {}

for value in data:
    
    key = cif_entries[value]

    aux_dict={}

    aux_dict['name']=key['_chem_comp']['name']
    aux_dict['three_letter_code']=key['_chem_comp']['three_letter_code']
    aux_dict['atom_name']=[key['_chem_comp_atom']['atom_id']]
    aux_dict['atom_name'].append(key['_chem_comp_atom']['alt_atom_id'])
    if '_chem_comp_bond' in key:
        bonds = []
        for ii,jj in zip(key['_chem_comp_bond']['atom_id_1'], key['_chem_comp_bond']['atom_id_2']):
            iii = aux_dict['atom_name'][0].index(ii)
            jjj = aux_dict['atom_name'][0].index(jj)
            if iii>jjj:
                bonds.append([jjj,iii])
            else:
                bonds.append([iii,jjj])
        aux_dict['bonds']=bonds
    else:
        aux_dict['bonds']=[]

    output[value]=aux_dict

In [4]:
path = '/usr/local/gromacs/share/gromacs/top/residues.xml'

aux_dict_2={}

with open(path, 'r') as fff:

    for line in fff.readlines():

        if '<residue restype' in line:
            resname = line.split('"')[1]
            aux_dict_2[resname]={'atoms':[], 'bonds':[]}
        elif '<ratom name' in line:
            iii = line.split('"')
            aux_dict_2[resname]['atoms'].append(iii[1])
        elif '<rbond a1' in line:
            iii =  line.split('"')
            aux_dict_2[resname]['bonds'].append([iii[1], iii[3]])

In [20]:
aux=[]
for ii in aux_dict_2:
    if ii not in output:
        aux.append(ii)

In [63]:
resname = 'SER'

import networkx as nx

A = nx.Graph()
B = nx.Graph()

for bond in output[resname]['bonds']:
    A.add_edge(bond[0], bond[1])

for bond in aux_dict_2[resname]['bonds']:
    B.add_edge(bond[0], bond[1])


In [64]:
B.remove_node('-C')

In [65]:
matcher = nx.algorithms.isomorphism.GraphMatcher(A, B)

In [66]:
is_subgraph_isomorphic = matcher.subgraph_is_isomorphic()
print("A contiene un subgrafo isomorfo a B:", is_subgraph_isomorphic)

A contiene un subgrafo isomorfo a B: True


In [67]:
from molsysmt.element.atom.get_atom_type import _get_atom_type_from_atom_name

In [71]:
if is_subgraph_isomorphic:
    eq_max=0
    candidato=None
    for mapping in matcher.subgraph_isomorphisms_iter():
        mask=[]
        eq=0
        for ii,jj in mapping.items():
            iii = _get_atom_type_from_atom_name(output[resname]['atom_name'][0][ii])
            jjj = _get_atom_type_from_atom_name(jj)
            mask.append(iii==jjj)
            if output[resname]['atom_name'][0][ii]==jj:
                eq+=1
        print(mapping, eq)
        if all(mask):
            if eq_max<eq:
                eq_max=eq
                candidato=mapping

{0: 'N', 8: 'H', 1: 'CA', 9: 'HA', 4: 'CB', 2: 'C', 10: 'HB1', 11: 'HB2', 5: 'OG', 12: 'HG', 3: 'O'} 8
{0: 'N', 8: 'H', 1: 'CA', 9: 'HA', 4: 'CB', 2: 'C', 10: 'HB1', 11: 'HB2', 5: 'OG', 12: 'HG', 6: 'O'} 7
{0: 'N', 8: 'H', 1: 'CA', 9: 'HA', 4: 'CB', 2: 'C', 11: 'HB1', 10: 'HB2', 5: 'OG', 12: 'HG', 3: 'O'} 9
{0: 'N', 8: 'H', 1: 'CA', 9: 'HA', 4: 'CB', 2: 'C', 11: 'HB1', 10: 'HB2', 5: 'OG', 12: 'HG', 6: 'O'} 8
{0: 'N', 7: 'H', 1: 'CA', 9: 'HA', 4: 'CB', 2: 'C', 10: 'HB1', 11: 'HB2', 5: 'OG', 12: 'HG', 3: 'O'} 9
{0: 'N', 7: 'H', 1: 'CA', 9: 'HA', 4: 'CB', 2: 'C', 10: 'HB1', 11: 'HB2', 5: 'OG', 12: 'HG', 6: 'O'} 8
{0: 'N', 7: 'H', 1: 'CA', 9: 'HA', 4: 'CB', 2: 'C', 11: 'HB1', 10: 'HB2', 5: 'OG', 12: 'HG', 3: 'O'} 10
{0: 'N', 7: 'H', 1: 'CA', 9: 'HA', 4: 'CB', 2: 'C', 11: 'HB1', 10: 'HB2', 5: 'OG', 12: 'HG', 6: 'O'} 9
{2: 'N', 3: 'H', 1: 'CA', 9: 'HA', 4: 'CB', 0: 'C', 10: 'HB1', 11: 'HB2', 5: 'OG', 12: 'HG', 7: 'O'} 5
{2: 'N', 3: 'H', 1: 'CA', 9: 'HA', 4: 'CB', 0: 'C', 10: 'HB1', 11: 'HB2'

In [69]:
candidato

{0: 'N',
 7: 'H',
 1: 'CA',
 9: 'HA',
 4: 'CB',
 2: 'C',
 11: 'HB1',
 10: 'HB2',
 5: 'OG',
 12: 'HG',
 3: 'O'}

In [70]:
output[resname]['atom_name'][0]

['N',
 'CA',
 'C',
 'O',
 'CB',
 'OG',
 'OXT',
 'H',
 'H2',
 'HA',
 'HB2',
 'HB3',
 'HG',
 'HXT']